We are interested in the expression and haplotype of a particular ZP3 gene (LOC115551355 in the Atlantic cod genome gadMor3.0), as it has previously been implicated as under selection and there is distinct spatial structure in Alaskan waters. We have expression data for this ZP3 gene, which show higher levels in cold (3C) compared to control (6C) and warm temperature (10C). Expression level also decreases linearly with increasing temperature. 

![zp3-boxplot](../results/ZP3-boxplot.jpg)
![zp3-linear](../results/ZP3-linear-models.jpg)

We want to identify the ZP3 haplotype of our experimental P. cod larvae from the RNASeq data. In this notebook I will explore the alignment data for the region containing the ZP3 gene only. 

First I will see about pulling alignment data for only the ZP3 gene. Where is that gene? 
```
(base) [lspencer@sedna GCF_902167405.1]$ cat gadMor3.0_genes.bed | grep "115551355"
NC_044056.1     2454600 2458007 gene-LOC115551355;Dbxref=GeneID:115551355;Name=LOC115551355;gbkey=Gene;gene=LOC115551355;gene_biotype=protein_coding
```

The ZP3 gene is on the RefSeq "NC_044056.1", which is chromosome 9 on the [Atlantic cod genome](https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_902167405.1/). It spans from  2454600-2458007. Let's try to extract alignment data for just that region: 


```
SAMPLES=$(ls *.Aligned.sortedByCoord.out.bam)

for sample in ${SAMPLES}
do

# First index all .bam files
samtools index ${sample}
done 
```

The VPN was terminated midway, so I reran the above code on all samples starting from PCG157 using this code:

```
SAMPLES=$(ls *.Aligned.sortedByCoord.out.bam)
startfile="PCG353.Aligned.sortedByCoord.out.bam"
for FILE in ${SAMPLES}; do
    if ! [ "$FILE" '<' "$startfile" ] ; then
        echo indexing $FILE
        samtools index $FILE
    else
        echo Skipping $FILE
    fi
done
```

??NC_044056.1:2,454,259 - 2,458,347??
Now extract just ZP3 gene region, and index that new bam file 
```
SAMPLES=$(ls *.Aligned.sortedByCoord.out.bam | sed -e 's/.Aligned.sortedByCoord.out.bam//')

for sample in ${SAMPLES}
do
samtools view -b -h ${sample}.Aligned.sortedByCoord.out.bam "NC_044056.1:2454600-2458007" > ${sample}.ZP3.bam
samtools sort ${sample}.ZP3.bam > ${sample}.ZP3.sorted.bam
samtools index ${sample}.ZP3.sorted.bam
done
```

#### What I actually did: 

Pull alignment files for the ZP3 plus/minus 2kb
gene location on complement strand (i.e. minus strand): 2454600 - 2458007
pull this range: 2452600-2460007

```
SAMPLES=$(ls *.Aligned.sortedByCoord.out.bam | sed -e 's/.Aligned.sortedByCoord.out.bam//')

for sample in ${SAMPLES}
do
samtools view -b -h ${sample}.Aligned.sortedByCoord.out.bam "NC_044056.1:2452600-2460007" > ${sample}.zp3.bam
samtools sort ${sample}.zp3.bam > ${sample}.zp3.sorted.bam
samtools index ${sample}.zp3.sorted.bam
done
```

I could use IGV to pull a consensus sequence for each sample individuall, which would be a pain and not super accurate since samples vary considerably in their coverage. So, instead I'll merge alignment data from all samples into one file, then view that in IGV and pull a consensus sequence for the ZP3 gene from all samples. 
```
samtools merge merged.zp3.bam *.zp3.sorted.bam
samtools sort merged.zp3.bam > merged.zp3.sorted.bam
samtools index merged.zp3.sorted.bam
```


Nice code to show the last modification date/time: 
```
bams=$(ls *.chr9.sorted.bam)
for file in ${bams}
do
echo ${file}
date -r ${file}
done
```